Silver Transformation - Data

In [0]:
bronzepath = "abfss://bronze@netflixprojdlake.dfs.core.windows.net/"
silverpath = "abfss://silver@netflixprojdlake.dfs.core.windows.net/"
goldpath = "abfss://gold@netflixprojdlake.dfs.core.windows.net/"


In [0]:
arrsyValues = [
    {
      "sourcefolder":"netflix_titles"  ,
      "targetfolder":"netflix_titles"
    },
    {
      "sourcefolder":"netflixcast"  ,
      "targetfolder":"netflixcast"
    },
    {
      "sourcefolder":"netflixcategory"  ,
      "targetfolder":"netflixcategory"
    },
    {
      "sourcefolder":"netflixcountries"  ,
      "targetfolder":"netflixcountries"
    },
    {
      "sourcefolder":"netflixdirectors"  ,
      "targetfolder":"netflixdirectors"
    }
]

In [0]:
# for i in range(0,len(arrsyValues)):
#     df_location = spark.read.format("csv") \
#     .option("header", "true") \
#     .option("inferSchema", "true") \
#     .load(bronzepath + f"/{arrsyValues[i]['sourcefolder']}/")  
#     display(df_location)

In [0]:
from pyspark.sql.types import *

schematitles = StructType([
    StructField("duration_minutes", IntegerType(), True),
    StructField("duration_seasons", IntegerType(), True),
    StructField("type", StringType(), True),
    StructField("title", StringType(), True),
    StructField("date_added", StringType(), True),
    StructField("release_year", IntegerType(), True),
    StructField("rating", StringType(), True),
    StructField("description", StringType(), True),
    StructField("show_id", IntegerType(), True)])

df_netflix_titles = spark.read.format("csv") \
    .option("header", "true") \
    .schema(schematitles) \
    .load(bronzepath + "/netflix_titles/")  

display(df_netflix_titles)



In [0]:
df_netflix_titles = df_netflix_titles.fillna({"duration_minutes":"0", "duration_seasons":"1"})

In [0]:
display(df_netflix_titles)

In [0]:
df_netflix_titles.createOrReplaceTempView("df_netflix_titles")
# df_netflix_titles = spark.sql("

In [0]:
# %sql
# SELECT *
# FROM df_netflix_titles
# WHERE try_cast(duration_minutes AS DOUBLE) IS NULL;

In [0]:
# %sql
# UPDATE df_netflix_titles
# SET
#     duration_minutes = CAST(CASE WHEN try_cast(duration_minutes AS DOUBLE) IS NULL THEN 0 ELSE duration_minutes END AS DOUBLE),
#     duration_seasons = CAST(CASE WHEN try_cast(duration_seasons AS DOUBLE) IS NULL THEN 0 ELSE duration_seasons END AS DOUBLE),
#      date_added = COALESCE(try_cast(date_added AS DATE), DATE '1900-01-01')

In [0]:
# %sql
# CREATE OR REPLACE TABLE df_netflix_titles AS
# SELECT 
#     CAST(CASE WHEN try_cast(duration_minutes AS DOUBLE) IS NULL THEN 0 ELSE duration_minutes END AS DOUBLE) AS duration_minutes,
#     CAST(CASE WHEN try_cast(duration_seasons AS DOUBLE) IS NULL THEN 0 ELSE duration_seasons END AS DOUBLE) AS duration_seasons,
#     COALESCE(try_cast(date_added AS DATE), DATE '1900-01-01') AS date_added,
#     *
# EXCEPT(duration_minutes, duration_seasons, date_added)
# FROM df_netflix_titles

In [0]:
# %sql
# select * from df_netflix_titles
# WHERE try_cast(duration_minutes AS DOUBLE) IS NULL;

In [0]:
from pyspark.sql.functions import *

df_netflix_titles = df_netflix_titles.withColumn("shortTitle", split(col("title"), ":")[0])

In [0]:
display(df_netflix_titles)

In [0]:
df_netflix_titles = df_netflix_titles.withColumn("rating", split(col("rating"), '-')[0])

In [0]:
display(df_netflix_titles)

In [0]:
df_netflix_titles = df_netflix_titles.withColumn("typeFlag", when(col("type") == "TV Show", 1)
                             .when(col("type") == "Movie", 2)
                             .otherwise(0))

In [0]:
df_netflix_titles.show(2)

In [0]:
display(df_netflix_titles)

In [0]:
from pyspark.sql.window import Window

df_netflix_titles = df_netflix_titles.withColumn("durationRanking",dense_rank().over(Window.orderBy(col("duration_minutes").desc())))


df_netflix_titles.display()

In [0]:
df_netflix_titles.createOrReplaceTempView("df_netflix_titles")

In [0]:
%sql
select * from df_netflix_titles

In [0]:
from pyspark.sql.functions import count
dfNetflixTitlesgroup = df_netflix_titles.groupBy("type").agg(count("*").alias("typeCount"))

display(dfNetflixTitlesgroup)

In [0]:
dfNetflixTitlesgroup.write.format("delta").mode("overwrite").save(silverpath+"/netflix_titles/aggregatedData/")

In [0]:
df_netflix_titles.write.format("csv").mode("overwrite").save(silverpath+"/netflix_titles/csv/cleanData/")